In [18]:
import pandas as pd
import numpy as np
from ipumspy import IpumsApiClient, MicrodataExtract, readers, ddi
from Credentials import MyCredentials
from Functions import Paths as P
from pathlib import Path

DataDir = Path(P['data'])
AcsDir = Path(P['acs'])
AcsPreDir = Path(P['preperiod'])

# Connect to API
ipums = IpumsApiClient(MyCredentials['IpumsApiKey'])

# IPUMS Pull
See the [documentation](https://ipumspy.readthedocs.io/en/stable/getting_started.html) for how to use ipumspy.

# State Level Analysis Files
I pull the 2000 - 2022 ACS samples and save them as Stata datasets.

**Note:** The years 2023 and 2024 will be included in the analysis, but I use the CPS for those since, for some reason, ACS stops having all the variables we would like.

In [11]:
Vars = ['STATEFIP',        # Geographic
        'CITIZEN', 'BPL',  # Citizenship/nativity
        'AGE',             # Demographics
        'OCC','IND1990','INDNAICS','OCC1990', 'OCC2010', # Work
        'UHRSWORK',                        # Work
        'EDUC',                            # Education
        'INCWAGE'                          # Income
        ]
SampleList =  ['us' + str(year) + 'a' for year in range(2000,2023)] # Only going to 2022 because 2023 doesn't have all the variables we want

for samp in SampleList:
    print('Creating and Downloading Extract: ' + samp)
    extract = MicrodataExtract('usa', [samp], Vars)
    ipums.submit_extract(extract)
    ipums.wait_for_extract(extract)
    ipums.download_extract(extract,download_dir=AcsDir)

Creating and Downloading Extract: us2000a


KeyboardInterrupt: 

### Read In and Save the "Pre-Period" Data

In [19]:
Vars = ['STATEFIP',        # Geographic
        'BPL',             # Citizenship/nativity
        'AGE'              # Demographics 
        ]
SampleList = ['us' + str(year) + 'a' for year in range(1920,2000,10)]
for samp in SampleList:
    print('Creating and Downloading Extract: ' + samp)
    extract = MicrodataExtract('usa', [samp], Vars)
    ipums.submit_extract(extract)
    ipums.wait_for_extract(extract)
    ipums.download_extract(extract,download_dir=AcsPreDir)

# Construct file list
Files = list(AcsPreDir.glob("*.xml"))
for f in Files:

    # Get the Data dictionary
    ddi = readers.read_ipums_ddi(f)
    df = readers.read_microdata(ddi, AcsPreDir / ddi.file_description.filename)
    yyyystr = str(df['YEAR'][0])
    df.to_csv(P['preperiod'] + '/PrePeriod' + yyyystr + '.csv', index=False)
    

Creating and Downloading Extract: us1920a
Creating and Downloading Extract: us1930a
Creating and Downloading Extract: us1940a
Creating and Downloading Extract: us1950a
Creating and Downloading Extract: us1960a
Creating and Downloading Extract: us1970a
Creating and Downloading Extract: us1980a
Creating and Downloading Extract: us1990a


/home/r/remuszka/.conda/envs/ImmigrationTfp/lib/python3.12/site-packages/ipumspy/readers.py:70: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See the `ipums_conditions` attribute of this codebook for terms of use.
See the `ipums_citation` attribute of this codebook for the appropriate citation.
  warnings.warn(
/home/r/remuszka/.conda/envs/ImmigrationTfp/lib/python3.12/site-packages/ipumspy/readers.py:70: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See the `ipums_conditions` attribute of this codebook for terms of use.
See the `ipums_citation` attribute of this codebook for the appropriate citation.
  warnings.warn(
/home/r/remuszka/.conda/envs/ImmigrationTfp/lib/python3.12/site-packages/ipumspy/readers.py:70: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See 